In [1]:
library(scran)
library(ggplot2)
library(RColorBrewer)
library(MASS)
library(biomaRt)

Loading required package: BiocParallel
Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, 

In [9]:
do_correction <- function(atlas_path, experiment_path, prefix, separator){
        
    df2 = read.delim(atlas_path)
    df4 = read.delim(experiment_path, sep=separator, row.names=1)

    rownames(df2) = sub("_.*","",rownames(df2))
    #rownames(df4) = sub("_.*","",rownames(df4))
    df4 = df4[!grepl("_", rownames(df4)),]
    
        
    # пересечение дней 8.25 и атласа
    gene_list_intersecton42 = intersect(rownames(df4), rownames(df2))
    length(gene_list_intersecton42)
    
    #порезанные файлы для введения MNN коррекции
    df2_cutted = df2[gene_list_intersecton42,] # атлас
    df4_cutted = df4[gene_list_intersecton42,] # 8.25
    
    print(table(rownames(df2_cutted) == rownames(df4_cutted)))
    
    # корректировка - 8.25 день (df4)
    print('Now its mNN')
    mNN_3 = mnnCorrect(as.matrix(df2_cutted), as.matrix(df4_cutted))
    
    df2_cutted_corrected = mNN_3$corrected[[1]]
    df4_cutted_corrected = mNN_3$corrected[[2]]
    
    # assigning right colnames and rownames
    colnames(df2_cutted_corrected) = colnames(df2_cutted)
    rownames(df2_cutted_corrected) = rownames(df2_cutted)

    colnames(df4_cutted_corrected) = colnames(df4_cutted)
    rownames(df4_cutted_corrected) = rownames(df4_cutted)
    
    path_head = '~/chukreev/batch_effect'
    print('Writing to files')
    write.table(df2_cutted_corrected, file=file.path(PATH=path_head, paste0("atlas_", prefix, "_corrected.tsv")), sep="\t", row.names = TRUE, quote = FALSE, col.names = TRUE)
    write.table(df4_cutted_corrected, file=file.path(PATH=path_head, paste0("experiment_", prefix, "_corrected.tsv")), sep="\t", row.names = TRUE, quote = FALSE, col.names = TRUE)

}

In [6]:
atlas_path = '/home/churkreev_group/references/7_25/counts_E_7_25.tsv'
experiment_path = '/home/churkreev_group/exp_data/7_25/Badabam_expression_matrix_annotated_genes.tsv'
prefix = '75'

In [7]:
df2 = read.delim(atlas_path)
df4 = read.delim(experiment_path, sep='\t', row.names=1)

In [8]:
dim(df2)
dim(df4)

[1] 29452 15294

[1] 12792     0

In [26]:
df4_cutted_corrected = read.delim("~/chukreev/batch_effect/experiment_6_5_corrected_1.tsv")

In [3]:
# 6_5
do_correction('/home/churkreev_group/references/6_5/counts_E_65.tsv', 
             '/home/churkreev_group/exp_data/6_5/day6.5',
             '6_5')


 TRUE 
11037 


In [10]:
# 7_25
do_correction('/home/churkreev_group/references/7_25/counts_E_7_25.tsv', 
             '/home/churkreev_group/exp_data/7_25/Badabam_expression_matrix_annotated_genes.tsv',
             '725',
             separator=',')


 TRUE 
11871 


In [11]:
# 8_25
do_correction('/home//churkreev_group/references/8_25/counts_E_8_25.tsv', 
             '/home/churkreev_group/exp_data/8_25/IGotNoBoobs_expression_matrix.tsv',
             '825')

ERROR: Error in read.delim(experiment_path, sep = separator, row.names = 1): argument "separator" is missing, with no default
